## Projet Deep Learning : Reconnaissance d'images de pays
## **Preprocessing**

#### Dataset: 
Google Landmarks Dataset v2 :
https://github.com/cvdfoundation/google-landmark?tab=readme-ov-file


#### Objectif du projet :
<p style="text-align: justify;">
    L'objectif de ce projet est de développer une solution de Deep Learning pour la reconnaissance d'images. L'input sera une image d'un lieu, et la sortie du modèle sera le pays correspondant, accompagnée de probabilités d'appartenance.
</p>


In [17]:
# Packages
import pandas as pd
import matplotlib.pyplot as plt


In [18]:
# Variables globales
DATA_TRAIN_PATH = "data/train"
DATA_IMAGES_TRAIN_PATH = "data/train/images"

### Importation des données

In [19]:
train_df = pd.read_csv(f"{DATA_TRAIN_PATH}/train.csv")
train_df.head()

,id,url,landmark_id
0,6e158a47eb2ca3f6,https://upload.wikimedia.org/wikipedia/commons...,142820
1,202cd79556f30760,http://upload.wikimedia.org/wikipedia/commons/...,104169
2,3ad87684c99c06e1,http://upload.wikimedia.org/wikipedia/commons/...,37914
3,e7f70e9c61e66af3,https://upload.wikimedia.org/wikipedia/commons...,102140
4,4072182eddd0100e,https://upload.wikimedia.org/wikipedia/commons...,2474


In [20]:
category_to_location_df = pd.read_csv(f"{DATA_TRAIN_PATH}/category_to_location.csv")
category_to_location_df.rename(columns={'id': 'landmark_id'}, inplace=True)
category_to_location_df.head()

,landmark_id,category_name,name,lat,lon,city,state,country
0,0,Category:Happy_Valley_Racecourse,Natural Turf Soccer Pitch No. 5,22.272800,114.182000,Hong Kong Island,Hong Kong,China
1,1,Category:Luitpoldpark_in_Munich,NaN,48.171494,11.569674,Munich,Bavaria,Germany
2,3,"Category:Tweed_Heads,_New_South_Wales",Ukerebagh Nature Reserve,-28.183300,153.550000,Tweed Heads,New South Wales,Australia
3,14,Category:Delacorte_Theater,Delacorte Theater,40.780100,-73.968767,New York,New York,United States
4,15,Category:Tremper_Mound_and_Earthworks,Tremper Mound,38.801300,-83.010600,NaN,Ohio,United States


### Jointure des données

On joint les lieux (ville, pays, ...) aux images

In [21]:
train_df = train_df.merge(category_to_location_df, left_on='landmark_id', right_on='landmark_id', how='left')
train_df.head()

,id,url,landmark_id,category_name,name,lat,lon,city,state,country
0,6e158a47eb2ca3f6,https://upload.wikimedia.org/wikipedia/commons...,142820,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202cd79556f30760,http://upload.wikimedia.org/wikipedia/commons/...,104169,Category:Stirling_Castle,Stirling Castle,56.123889,-3.947778,Stirling,Scotland,United Kingdom
2,3ad87684c99c06e1,http://upload.wikimedia.org/wikipedia/commons/...,37914,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,e7f70e9c61e66af3,https://upload.wikimedia.org/wikipedia/commons...,102140,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4072182eddd0100e,https://upload.wikimedia.org/wikipedia/commons...,2474,Category:River_Severn,Aylburton,51.685278,-2.543611,Forest of Dean,England,United Kingdom


In [22]:
print('shape du dataset de base : ',train_df.shape)
train_df.dropna(subset=['country'], inplace=True)
print('shape du dataset ne conservant que les lieux reconnus : ',train_df.shape)
train_df.head()

shape du dataset de base :  (4132914, 10)
shape du dataset ne conservant que les lieux reconnus :  (1273626, 10)


,id,url,landmark_id,category_name,name,lat,lon,city,state,country
1,202cd79556f30760,http://upload.wikimedia.org/wikipedia/commons/...,104169,Category:Stirling_Castle,Stirling Castle,56.123889,-3.947778,Stirling,Scotland,United Kingdom
4,4072182eddd0100e,https://upload.wikimedia.org/wikipedia/commons...,2474,Category:River_Severn,Aylburton,51.685278,-2.543611,Forest of Dean,England,United Kingdom
7,16d8aa057cdd01b9,http://upload.wikimedia.org/wikipedia/commons/...,25719,Category:Duomo_(Monza),Monza Cathedral,45.583590,9.275670,Monza,Lombardy,Italy
12,88f3f71c2b71a6f9,https://upload.wikimedia.org/wikipedia/commons...,198623,"Category:Newark_Castle,_Nottinghamshire",NaN,53.077500,-0.812415,Newark and Sherwood,England,United Kingdom
15,0851a257e5e872ef,https://upload.wikimedia.org/wikipedia/commons...,189446,Category:Castle_of_Peñíscola,Castillo de Peñiscola,40.358800,0.407926,Peníscola / Peñíscola,Valencian Community,Spain


Récupération des chemins des images

In [31]:
import importlib
import fetch_image
importlib.reload(fetch_image)

chemin_images_dict = fetch_image.fetch_images(train_df['id'], dossier_base = DATA_IMAGES_TRAIN_PATH)

Ajout des chemins au df train

In [35]:
train_df['image_path'] = train_df['id'].map(chemin_images_dict)
print('shape du dataset ne conservant que les lieux reconnu : ',train_df.shape)
train_df.dropna(subset=['image_path'], inplace=True)
print('shape du dataset ne conservant que les lieux reconnus et images trouvées dans les dossiers : ',train_df.shape)
train_df.head()

shape du dataset ne conservant que les lieux reconnu :  (1273626, 11)
shape du dataset ne conservant que les lieux reconnus et images trouvées dans les dossiers :  (15264, 11)


,id,url,landmark_id,category_name,name,lat,lon,city,state,country,image_path
172,00c08b162f34f53f,https://upload.wikimedia.org/wikipedia/commons...,163404,Category:North_Norfolk_Railway,Weybourne Yard Frame,52.934500,1.154500,North Norfolk,England,United Kingdom,data\train\images\0\0\c\00c08b162f34f53f.jpg
682,0129308917af0393,https://upload.wikimedia.org/wikipedia/commons...,20823,"Category:Westmoreland_County,_Pennsylvania",Dellview Court,40.310000,-79.470000,Unity Township,Pennsylvania,United States,data\train\images\0\1\2\0129308917af0393.jpg
710,00e5d77c905d94a6,https://upload.wikimedia.org/wikipedia/commons...,26066,Category:Santuário_de_Fátima,Basílica de Nossa Senhora do Rosário de Fátima,39.632427,-8.671538,Fátima,NaN,Portugal,data\train\images\0\0\e\00e5d77c905d94a6.jpg
1180,0270b8d88aca27c4,https://upload.wikimedia.org/wikipedia/commons...,181586,Category:HMCS_Haida_(G63),HMCS Haida,43.275300,-79.855400,Hamilton,Ontario,Canada,data\train\images\0\2\7\0270b8d88aca27c4.jpg
1262,001cd787f1e9a803,https://upload.wikimedia.org/wikipedia/commons...,61937,Category:South_Horizons,HK Electric Co. Ltd. Operational HQ,22.243364,114.147564,Hong Kong Island,Hong Kong,China,data\train\images\0\0\1\001cd787f1e9a803.jpg


### Transformation des images en tenseurs

Test sur une image

In [ ]:
import torch
from PIL import Image
from torchvision import transforms

def image_to_tensor(image_path):
    """
    charge une image jpg et convertit en tenseur pytorch
    applique des transformations sur le tenseur
    
    Args:
        image_path (str): chemin de l'image jpg
    
    Returns:
        torch.Tensor: tesneur pytorch
    """

    image = Image.open(image_path)
    
    # transformation
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # taille standard
        transforms.ToTensor(),          # conversion des valeurs entre 0 et 1
        transforms.Normalize(           # normalisation
            mean=[0.485, 0.456, 0.406],  # moyenne RGB pour ImageNet
            std=[0.229, 0.224, 0.225]     # std RGB pour ImageNet
        )
    ])
    
    tensor = transform(image)
    
    # ajout une dimension de batch
    tensor = tensor.unsqueeze(0)
    
    return tensor


In [ ]:
image_to_tensor(f'{DATA_IMAGES_TRAIN_PATH}/0/0/0/000a0aee5e90cbaf.jpg')

tensor([[[[ 0.0227,  0.0227,  0.0569,  ...,  0.1083,  0.1083,  0.1254],
          [ 0.0227,  0.0227,  0.0569,  ...,  0.1254,  0.1254,  0.1254],
          [ 0.0227,  0.0227,  0.0569,  ...,  0.1254,  0.1254,  0.1426],
          ...,
          [-1.6384, -1.6555, -1.6384,  ...,  1.0844,  1.1015,  1.0844],
          [-1.6384, -1.6384, -1.6384,  ...,  1.0844,  1.1015,  1.0844],
          [-1.6213, -1.6213, -1.6213,  ...,  1.1015,  1.1015,  1.1358]],

         [[ 0.6429,  0.6429,  0.6604,  ...,  0.7304,  0.7304,  0.7479],
          [ 0.6429,  0.6429,  0.6604,  ...,  0.7479,  0.7479,  0.7479],
          [ 0.6429,  0.6429,  0.6604,  ...,  0.7479,  0.7479,  0.7654],
          ...,
          [-1.8256, -1.8256, -1.8256,  ...,  1.0105,  1.0280,  1.0105],
          [-1.8081, -1.8081, -1.8256,  ...,  1.0105,  1.0280,  1.0105],
          [-1.7906, -1.7906, -1.7906,  ...,  1.0280,  1.0280,  1.0630]],

         [[ 1.5420,  1.5420,  1.5245,  ...,  1.6291,  1.6291,  1.6465],
          [ 1.5420,  1.5420,  